In [3]:
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

def set_cpu_num(cpu_num):
    import os
    import torch
    if cpu_num <= 0: return
    
    os.environ ['OMP_NUM_THREADS'] = str(cpu_num)
    os.environ ['OPENBLAS_NUM_THREADS'] = str(cpu_num)
    os.environ ['MKL_NUM_THREADS'] = str(cpu_num)
    os.environ ['VECLIB_MAXIMUM_THREADS'] = str(cpu_num)
    os.environ ['NUMEXPR_NUM_THREADS'] = str(cpu_num)
    torch.set_num_threads(cpu_num)
set_cpu_num(10)

In [4]:
import torch
import numpy as np


x_train = np.load(f'/data5/chengjingwen/lo/uv.npy')
x_train = torch.tensor(x_train, dtype=torch.float32)  
print(x_train.shape)
num_trajectories, total_steps, num_var, x_dim, y_dim = x_train.shape

xmin = x_train.amin(dim=(0, 1, 3, 4), keepdim=True)  
xmax = x_train.amax(dim=(0, 1, 3, 4), keepdim=True)  


data = (x_train - xmin) / (xmax - xmin)


print(f"Training data normalized: min={data.min().item()}, max={data.max().item()}")

num_train = int(num_trajectories * 0.8)  # 80%的轨迹数
random_indices = torch.randperm(num_trajectories)[:num_train]  
data = data[random_indices]
num_trajectories, total_steps, num_var, x_dim, y_dim = data.shape


torch.Size([100, 100, 2, 64, 64])
Training data normalized: min=0.0, max=1.0


In [5]:
class SingleStepDataset(Dataset):
    def __init__(self, data, pred_step):
        """
        data: Tensor, shape (num_trajectories, num_steps, num_variables, x_dim, y_dim)
        """
        self.data = data
        self.num_trajectories, self.num_steps, self.num_variables, self.x_dim, self.y_dim = data.shape
        self.pred_step = pred_step
       

    def __len__(self):
        # 每条轨迹生成 num_steps // self.pred_step + 1 个样本
        return self.num_trajectories * (self.num_steps // (self.pred_step + 1))

    def __getitem__(self, idx):
        
        trajectory_idx = idx // (self.num_steps // (self.pred_step + 1))
        time_idx = (idx % (self.num_steps // (self.pred_step + 1))) * (self.pred_step + 1)  

        
        input_data = self.data[trajectory_idx, time_idx]  # (num_variables, x_dim, y_dim)

       
        target_data = self.data[trajectory_idx, time_idx + 1 : time_idx+self.pred_step+1]  # (num_variables, x_dim, y_dim)

        return input_data, target_data

train_data = data
pred_step = 5

train_dataset = SingleStepDataset(train_data, pred_step = pred_step)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


for inputs, targets in train_loader:
    print(f"Input shape: {inputs.shape}, Target shape: {targets.shape}")
    num_variables = inputs.shape[1]
    break  # 仅打印第一个 batch 的信息

Input shape: torch.Size([32, 2, 64, 64]), Target shape: torch.Size([32, 5, 2, 64, 64])


In [6]:
import torch
import torch.nn as nn
from collections import OrderedDict

class UNet2d(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, init_features=32, pred_step=5):
        super().__init__()

        features = init_features
        self.encoder1 = UNet2d._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNet2d._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNet2d._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = UNet2d._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bottleneck = UNet2d._block(features * 8, features * 16, name="bottleneck")

        self.upconv4 = nn.ConvTranspose2d(
            features * 16, features * 8, kernel_size=2, stride=2
        )
        self.decoder4 = UNet2d._block((features * 8) * 2, features * 8, name="dec4")
        self.upconv3 = nn.ConvTranspose2d(
            features * 8, features * 4, kernel_size=2, stride=2
        )
        self.decoder3 = UNet2d._block((features * 4) * 2, features * 4, name="dec3")
        self.upconv2 = nn.ConvTranspose2d(
            features * 4, features * 2, kernel_size=2, stride=2
        )
        self.decoder2 = UNet2d._block((features * 2) * 2, features * 2, name="dec2")
        self.upconv1 = nn.ConvTranspose2d(
            features * 2, features, kernel_size=2, stride=2
        )
        self.decoder1 = UNet2d._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(
            in_channels=features, out_channels=out_channels, kernel_size=1
        )
        self.n = pred_step

    def _forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))

        bottleneck = self.bottleneck(self.pool4(enc4))

        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        return self.conv(dec1)
    
    def forward(self, x, n=None):
        if n is None:  # Use default n if not provided
            n = self.n
            
        y1 = self._forward(x)   # x: [batch_size, hidden_dim, input_dim, input_dim]
        y = [y1]
        
        # Multi-step prediction
        for _ in range(1, n):
            last_output = y[-1]  # [batch_size, hidden_dim, input_dim, input_dim]
            output = self._forward(last_output)   
            y.append(output)
        final_outputs = torch.stack(y, dim=1) # [batch_size, n, hidden_dim, input_dim, input_dim]
       
        # Return last prediction and all predictions
        return final_outputs  # [batch_size, n, num_var, input_dim, input_dim]

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "tanh1", nn.Tanh()),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "tanh2", nn.Tanh()),
                ]
            )
        )

In [7]:
# test
x = torch.rand((32, 2, 64, 64))
model = UNet2d(in_channels=2, out_channels=2, init_features=32, pred_step=5)
final_output = model(x)
num_params = sum(p.numel() for p in model.parameters())
print(f"模型总参数量: {num_params / 1e6:.2f} M")

模型总参数量: 7.76 M


In [8]:
print(final_output.shape)

torch.Size([32, 5, 2, 64, 64])


In [9]:
import torch
import torch.nn as nn
from torch.optim import Adam


input_step = 1
pred_step = 5
num_epochs = 100
batch_size = 64
device = "cuda:3" 

# 模型
model = UNet2d(in_channels=2, out_channels=2, init_features=32, pred_step=pred_step).to(device)

# 优化器
optimizer = Adam(list(model.parameters()), lr=1e-3)


criterion = nn.MSELoss()

model.train()
for epoch in range(num_epochs):
    total_loss = 0.0
    for batch_idx, (input_data, target_data) in enumerate(train_loader):
        input_data = input_data.to(device)  # Shape: [batch_size, num_variables, x_dim, y_dim]
        target_data = target_data.to(device)  # Shape: [batch_size, pred_step, num_variables, x_dim, y_dim]
        
        pred_data = model(input_data)
        loss = criterion(pred_data, target_data)
        total_loss += loss.item()

        optimizer.zero_grad()        
        loss.backward()
        optimizer.step()
        
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Pred Loss: {avg_loss:.5f}")
    

print("Training complete!")

Epoch [1/100], Pred Loss: 0.20459
Epoch [2/100], Pred Loss: 0.11358
Epoch [3/100], Pred Loss: 0.03553
Epoch [4/100], Pred Loss: 0.00996
Epoch [5/100], Pred Loss: 0.00717
Epoch [6/100], Pred Loss: 0.00391
Epoch [7/100], Pred Loss: 0.00352
Epoch [8/100], Pred Loss: 0.00332
Epoch [9/100], Pred Loss: 0.00320
Epoch [10/100], Pred Loss: 0.00267
Epoch [11/100], Pred Loss: 0.00229
Epoch [12/100], Pred Loss: 0.00272
Epoch [13/100], Pred Loss: 0.00326
Epoch [14/100], Pred Loss: 0.00359
Epoch [15/100], Pred Loss: 0.00239
Epoch [16/100], Pred Loss: 0.00254
Epoch [17/100], Pred Loss: 0.00230
Epoch [18/100], Pred Loss: 0.00217
Epoch [19/100], Pred Loss: 0.00205
Epoch [20/100], Pred Loss: 0.00166
Epoch [21/100], Pred Loss: 0.00236
Epoch [22/100], Pred Loss: 0.00249
Epoch [23/100], Pred Loss: 0.00210
Epoch [24/100], Pred Loss: 0.00186
Epoch [25/100], Pred Loss: 0.00231
Epoch [26/100], Pred Loss: 0.00201
Epoch [27/100], Pred Loss: 0.00173
Epoch [28/100], Pred Loss: 0.00196
Epoch [29/100], Pred Loss: 0.

In [10]:
import torch
import numpy as np


x_test = np.load(f'data/lo/uv_test.npy')
x_test = torch.tensor(x_test, dtype=torch.float32)
x_test_normalized = (x_test - xmin) / (xmax - xmin)

print(f"Test data normalized: min={x_test_normalized.min().item()}, max={x_test_normalized.max().item()}")

Test data normalized: min=-0.00329780881293118, max=1.0405967235565186


In [11]:
import numpy as np
import torch
from torchmetrics.functional import structural_similarity_index_measure as ssim
from tqdm import tqdm

import matplotlib.pyplot as plt
from IPython.display import clear_output



pred_steps = 99

def compute_nmse(predictions, targets):
    """计算 NMSE"""
    return torch.mean(((predictions - targets) ** 2)) / torch.mean((targets ** 2))

def compute_rmse(predictions, targets):
    """计算 RMSE"""
    mse = torch.mean((predictions - targets) ** 2)
    return torch.sqrt(mse)


def test_model(convlstm, test_data, device):
    convlstm.eval()

    num_trajectories, T, num_variables, x_dim, y_dim = test_data.shape

    nmse_list = []
    ssim_list = []
    rmse_list = []

    for trajectory_idx in tqdm(range(num_trajectories)):
        trajectory = test_data[trajectory_idx]  # shape [100, num_variables, x_dim, y_dim]
        start_time = 0
        targets = trajectory[start_time + 1:start_time + 1 + pred_steps].to(device)  
        predictions = model(trajectory[:1], n=pred_steps).squeeze(0) # shape: [pred_steps, num_variables, x_dim, y_dim]

        # if trajectory_idx == 0:
        #     for t in range(pred_steps):
        #         plt.figure(figsize=(12, 4))

        #         # Ground Truth
        #         plt.subplot(131)
        #         plt.title(f"Ground Truth (step={ t + 1})")
        #         plt.imshow(targets[t, 0].detach().cpu().numpy(), cmap="jet", vmin=-1, vmax=1)  # 第一个变量的真值
        #         plt.colorbar()

        #         # Prediction
        #         plt.subplot(132)
        #         plt.title("Prediction")
        #         plt.imshow(predictions[t, 0].detach().cpu().numpy(), cmap="jet", vmin=-1, vmax=1)  # 第一个变量的预测值
        #         plt.colorbar()

        #         # Difference and MSE
        #         mse = torch.mean((targets[t, 0] - predictions[t, 0]) ** 2).item()  # 保证两者在同一设备
        #         plt.subplot(133)
        #         plt.title(f"Difference (MSE={mse:.5f})")
        #         plt.imshow(torch.abs(targets[t, 0] - predictions[t, 0]).detach().cpu().numpy(), cmap="jet")
        #         plt.colorbar()

        #         plt.show()
        #         clear_output(wait=True)  # 显示当前时间步，清除上一时间步
            

      
        ssim_values = ssim(predictions, targets, data_range=1.0).item()  # SSIM
        nmse_trajectory = compute_nmse(predictions, targets).item()  # NMSE
        rmse_trajectory = compute_rmse(predictions, targets).item()  # RMSE

        nmse_list.append(nmse_trajectory)
        ssim_list.append(ssim_values)
        rmse_list.append(rmse_trajectory)
   
    nmse_mean, nmse_std = np.mean(nmse_list), np.std(nmse_list)
    ssim_mean, ssim_std = np.mean(ssim_list), np.std(ssim_list)
    rmse_mean, rmse_std = np.mean(rmse_list), np.std(rmse_list)

    return {
        "NMSE_mean": nmse_mean,
        "NMSE_std": nmse_std,
        "SSIM_mean": ssim_mean,
        "SSIM_std": ssim_std,
        "RMSE_mean": rmse_mean,
        "RMSE_std": rmse_std
    }


test_data = x_test_normalized.to(device)
results = test_model(model, test_data, device)
print(f"NMSE Mean: {results['NMSE_mean']}")
print(f"NMSE Std: {results['NMSE_std']}")
print(f"SSIM Mean: {results['SSIM_mean']}")
print(f"SSIM Std: {results['SSIM_std']}")
print(f"RMSE Mean: {results['RMSE_mean']}")
print(f"RMSE Std: {results['RMSE_std']}")

  0%|          | 0/50 [00:00<?, ?it/s]/data4/liruikun/.conda/envs/SimBackbone/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:70: FutureWarning: Importing `spectral_angle_mapper` from `torchmetrics.functional` was deprecated and will be removed in 2.0. Import `spectral_angle_mapper` from `torchmetrics.image` instead.
  _future_warning(
100%|██████████| 50/50 [00:14<00:00,  3.41it/s]

NMSE Mean: 0.18125465273857116
NMSE Std: 0.06355004722925543
SSIM Mean: 0.5855060821771622
SSIM Std: 0.08082063331799154
RMSE Mean: 0.16975272223353385
RMSE Std: 0.028388472743270377
